In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader

c:\Users\Asus\Desktop\rough\jgenai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader=PyPDFLoader('NIVASH__R.pdf')

In [3]:
docs=loader.load()

In [4]:
docs

[Document(metadata={'producer': 'iLovePDF', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-09-30T18:26:34+05:30', 'author': 'R, Nivash', 'moddate': '2025-09-30T12:59:10+00:00', 'source': 'NIVASH__R.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='NIVASH R \n    Coimbatore |         nivash2920@gmail.com | ✆ +91 94452 91158 |  nivash-r \n \nSUMMARY \nInnovative Software Engineer with a strong foundation in designing, optimizing, and automating data -driven workflows. \nRecognized for bridging data engineering with emerging AI practices to create intelligent, end-to-end systems that drive \nbusiness value. Skilled at transforming complex data into meaningful insights and ensuring that technical solutions align \nwith organizational goals in a fast-evolving technological landscape. Committed to delivering scalable and reliable \nsolutions that drive efficiency, strengthen workflows, and keep pace with rapidly changing business and technology \nenvironments. \n

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)

In [7]:
chunk=splitter.split_documents(docs)

In [8]:
chunk

[Document(metadata={'producer': 'iLovePDF', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-09-30T18:26:34+05:30', 'author': 'R, Nivash', 'moddate': '2025-09-30T12:59:10+00:00', 'source': 'NIVASH__R.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='NIVASH R \n    Coimbatore |         nivash2920@gmail.com | ✆ +91 94452 91158 |  nivash-r \n \nSUMMARY \nInnovative Software Engineer with a strong foundation in designing, optimizing, and automating data -driven workflows. \nRecognized for bridging data engineering with emerging AI practices to create intelligent, end-to-end systems that drive \nbusiness value. Skilled at transforming complex data into meaningful insights and ensuring that technical solutions align'),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-09-30T18:26:34+05:30', 'author': 'R, Nivash', 'moddate': '2025-09-30T12:59:10+00:00', 'source': 'NIVASH__R.pdf', 'total_pages': 2, 'page': 0, 'page_l

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [10]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Asus\AppData\Local\Temp\ipykernel_16368\3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# test_vec= embeddings.embed_query('list all the skills')
# print(len(test_vec))

384


In [12]:
from langchain_community.vectorstores import FAISS

In [13]:
db=FAISS.from_documents(chunk,embeddings)

In [26]:
print("FAISS index created with", db.index.ntotal, "vectors")

FAISS index created with 13 vectors


In [37]:
retriever = db.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2")

C:\Users\Asus\AppData\Local\Temp\ipykernel_16368\2984537256.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [43]:
from langchain_core.prompts import ChatPromptTemplate

In [53]:
template=  """
Use the following context to answer the question:
{context}
Question: {question}
"""
prompt=ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nUse the following context to answer the question:\n{context}\nQuestion: {question}\n'), additional_kwargs={})])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 8. Create the RAG chain using LCEL
rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(doc.page_content for doc in docs)), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
#✅ Simple analogy: You asked “BMW AI vision” → retriever finds 3 chunks → lambda joins them into a single paragraph for the LLM.
# This becomes input | to this

# 9. Query the RAG system
questions = [
    "list all the certifications?",
    "list all the poc worked on?",
    "Can this person work as junior genai developer?"
]

print("\n=== Querying the RAG System ===")
for question in questions:
    print(f"\nQuestion: {question}")
    answer = rag_chain.invoke(question)
    print(f"Answer: {answer}")

# 10. Save the vector store for future use
#vectorstore.save_local("faiss_index_hf")
print("\nVector store saved to 'faiss_index_hf'")


=== Querying the RAG System ===

Question: list all the certifications?
Answer: The human has the following certifications:

1. Google Associate Cloud Engineer - GCP
2. Full Stack Developer - Simplilearn
3. Python Programming - Udemy
4. Python Coding - HackerRank
5. Generative AI - Databricks
6. AWS Certified Data Engineer - AWS

Question: list all the poc worked on?
Answer: Based on the provided context, here is the list of POC (Proof-of-Concept) projects mentioned:

1. Recommendation Updates (POC Project) - Refined AI-generated recommendations using iterative prompt strategies.
2. Dialogue Flow & Intent Optimization – Internal CX Project (POC Project) - Improved chatbot performance by analyzing and creating intents.

Note that there are also mentions of other projects, but they seem to be part of the individual's education or certifications rather than POC projects specifically.

Question: Can this person work as junior genai developer?
Answer: Based on the provided context, it seem